# Neptune Quickstart

<a target="_blank" href="https://colab.research.google.com/github/neptune-ai/scale-examples/blob/main/how-to-guides/quickstart/notebooks/neptune_scale_quickstart.ipynb"> 
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/> 
</a>
<a target="_blank" href="https://github.com/neptune-ai/scale-examples/blob/main/how-to-guides/quickstart/notebooks/neptune_scale_quickstart.ipynb">
  <img alt="Open in GitHub" src="https://img.shields.io/badge/Open_in_GitHub-blue?logo=github&labelColor=black">
</a>
<a target="_blank" href="https://scale.neptune.ai/o/examples/org/quickstart/reports/9ea26258-2aed-4441-9b58-bab619215f6f">
  <img alt="Explore in Neptune" src="https://neptune.ai/wp-content/uploads/2024/01/neptune-badge.svg">
</a>
<a target="_blank" href="https://docs.neptune.ai/quickstart">
  <img alt="View tutorial in docs" src="https://neptune.ai/wp-content/uploads/2024/01/docs-badge-2.svg">
</a>


This guide shows how to:
- Install and configure Neptune
- Create a Neptune run
- Log configuration values and metrics to the run


## Install Neptune and dependencies

In [ ]:
! pip install -q -U neptune_scale "numpy<2.0" tqdm Pillow torch torchvision

## Training boilerplate

Create dummy training, validation, and test step functions that return evaluation metrics

In [ ]:
NUM_STEPS = 20_000  # Determines how long the training will run for

In [ ]:
import numpy as np
from random import randint


def get_gradient_norm(layer: int, step: int) -> float:
    time_decay = 1.0 / (1.0 + step / 1000)
    layer_factor = np.exp(-0.5 * ((layer - 5) ** 2) / 4)
    noise = np.random.uniform(-0.1, 0.1) * (1 - step / NUM_STEPS)

    return (0.5 + layer_factor) * time_decay + noise


def get_gpu_utilization(step: int) -> float:
    base_util = 0.85
    data_loading_drop = 0.2 if step % 10 == 0 else 0.0
    update_spike = 0.1 if step % 5 == 0 else 0.0
    noise = np.random.uniform(-0.05, 0.05)

    return base_util - data_loading_drop + update_spike + noise


def _generate_metric(
    step: int,
    factor: float = 1.0,
) -> float:
    relative_progress = step / NUM_STEPS
    noise = np.random.uniform(-0.3, 0.3) * (1 - relative_progress)
    random_int = randint(0, 1000)

    return 1 / np.log(relative_progress / factor * random_int + 1.1) + noise


def training_step(step: int) -> tuple[float, float]:
    accuracy = 0.45 + 1 / (1 + np.exp(_generate_metric(step)))
    loss = _generate_metric(step)
    return accuracy, loss


def validation_step(step: int) -> tuple[float, float]:
    accuracy = 0.45 + 1 / (1 + np.exp(_generate_metric(step, 20)))
    loss = _generate_metric(step, 20)
    return accuracy, loss


def test_step(step: int) -> tuple[float, float]:
    accuracy = 0.45 + 1 / (1 + np.exp(_generate_metric(step, 30)))
    loss = _generate_metric(step, 30)
    return accuracy, loss

## Get and set your API token

If you haven't already, [create a project](https://docs.neptune.ai/projects).

To find your API token and full project name:
1. Log into Neptune.
2. In the bottom-left corner, expand your user menu and select **Get your API token**.
3. Copy the API token and paste it below to set as an environment variable. Uncomment the code before running.
4. To find the full project name, open the project settings. Copy and paste the project path below.


In [ ]:
# Set Neptune credentials as environment variables
# %env NEPTUNE_API_TOKEN = "YOUR_API_TOKEN"
# %env NEPTUNE_PROJECT = "YOUR_WORKSPACE_NAME/YOUR_PROJECT_NAME"

## Initialize the Neptune run

A [run](https://docs.neptune.ai/runs) is the basic unit of tracking in Neptune.

Initialize the `Run` object to start logging to Neptune. You can also pass your API token and project name as arguments to the `Run` constructor, but we recommend setting these as environment variables for security reasons.

You can also pass an optional `experiment_name` to organize runs into [experiments](https://docs.neptune.ai/experiments).

```python
run = Run(
    project = "YOUR_WORKSPACE_NAME/YOUR_PROJECT_NAME",  # Required if not set as environment variable
    api_token = "YOUR_API_TOKEN",  # Required if not set as environment variable
    experiment_name = "EXPERIMENT_NAME",  # Optional
    run_id = "UNIQUE_RUN_IDENTIFIER"  # Optional: Will be generated automatically if not specified
)
```

In addition to the above parameters, the `Run` constructor also accepts many other optional parameters that let you control the run's behavior in detail. You can explore them in the [API reference](https://docs.neptune.ai/run#parameters).

In [ ]:
from neptune_scale import Run

run = Run(experiment_name="quickstart-experiment")

print(f"Neptune run created 🎉\nAccess at {run.get_run_url()}")

The link to the run will take you to the run's page in the Neptune web app. It will mostly be empty for now, but will start filling up as we log data in the next steps.

## Add tags to identify and organize your runs

To add tags to your run, use the `add_tags()` method. You can pass a list, set, or tuple of strings as tags.
To add group tags, set the `group_tags` parameter to `True`.

In [ ]:
run.add_tags(["quickstart", "notebook"])
run.add_tags(["long"], group_tags=True)

## Log configuration parameters

To log any scalar values, use the `log_configs()` method.


In [ ]:
run.log_configs(
    {
        "parameters/data/use_preprocessing": True,
        "parameters/data/batch_size": 128,
        "parameters/model/activation": "relu",
        "parameters/model/dropout": 0.1,
        "parameters/optimizer/type": "Adam",
        "parameters/optimizer/learning_rate": 0.001,
    }
)

The above cell creates a _parameters_ folder in the run's _All Metadata_ tab. Within the _parameters_ folder, there are separate folders for _data_, _model_, and _optimizer_, and the configuration parameters are added to the respective folders.

This way, you can organize your configurations in a way that is easy to understand and navigate.


## Execute training loop that logs to Neptune

To log metric series to Neptune, use the `log_metrics()` method.

In [ ]:
from tqdm.auto import trange

for step in trange(NUM_STEPS):
    train_accuracy, train_loss = training_step(step)
    valid_accuracy, valid_loss = validation_step(step)
    test_accuracy, test_loss = test_step(step)

    # Collect metrics to log in a dictionary. The keys determine the attribute name and folder structure on Neptune
    metrics_to_log = {
        "metrics/train/accuracy": train_accuracy,
        "metrics/train/loss": train_loss,
        "metrics/valid/accuracy": valid_accuracy,
        "metrics/valid/loss": valid_loss,
        "metrics/test/accuracy": test_accuracy,
        "metrics/test/loss": test_loss,
    }

    # You can also log granular details, like per-layer gradient_norms, or per-GPU utilization at each training step
    for layer in range(10):
        metrics_to_log[f"debug/gradient_norm/layer_{layer}"] = get_gradient_norm(layer, step)
        metrics_to_log[f"system/gpu_{layer}/utilization"] = get_gpu_utilization(step)

    # Pass the `metrics_to_log` dictionary to the `log_metrics()` method to log all metrics at once
    run.log_metrics(
        data=metrics_to_log,
        step=step,
    )

print(f"Training complete ✅\nView charts at {run.get_run_url()}&detailsTab=charts")

The above code cell logs evaluation metrics under a `metrics` folder, further subdivided into `train`, `valid`, and `test` for those steps respectively. It also logs per-layer gradient norms and per-GPU utilization at each training step.

All of these metrics are logged as series, which means that they are logged as time series data. This allows you to see the evolution of the metrics over time. 

These metrics are also visualized as [Charts](https://docs.neptune.ai/charts/) in the Neptune web app. All charts update live and are visible in the **Charts** tab.


## Log single and series of files to Neptune

You can upload files individually or as a series with steps. For details, see [Upload files](https://docs.neptune.ai/upload_files) in the Neptune docs.

In [ ]:
# Download a sample image
! curl -o sample.png https://neptune.ai/wp-content/uploads/2024/05/blog_feature_image_046799_8_3_7_3-4.jpg
! curl -o sac-rl.mp4 https://neptune.ai/wp-content/uploads/2025/05/sac-rl.mp4
! curl -o t-rex.mp3  https://neptune.ai/wp-content/uploads/2025/05/t-rex.mp3

Use the `assign_files()` method to upload single files. You can:
- Log single or multiple files in one call
- Organize files in folders

In [ ]:
# Log a single file
run.assign_files(
    {
        "files/single/image": "sample.png",
        "files/single/video": "sac-rl.mp4",
        "files/single/audio": "t-rex.mp3",
    }
)

To log a sequence of files to a single attribute, use the `log_files()` method. For details, see [Upload a file series](https://docs.neptune.ai/upload_files#file-series) in the Neptune docs.

In [ ]:
from PIL import Image
import numpy as np


def tensor_to_pil_image(tensor):
    """
    Convert a PyTorch tensor to PIL Image format.
    """
    img_np = tensor.squeeze().numpy()
    img_np = (img_np * 255).astype(np.uint8)

    return Image.fromarray(img_np)

In [ ]:
# Download MNIST dataset
import torch
from torchvision import datasets, transforms

train_dataset = datasets.MNIST(
    root="./data", train=True, download=True, transform=transforms.Compose([transforms.ToTensor()])
)

# Upload a series of files to Neptune
for step in range(1, 10):
    img, label = train_dataset[step]
    pil_img = tensor_to_pil_image(img)
    # Save image to disk before upload
    pil_img.save(f"sample_{step}_label_{label}.png")

    run.log_files(
        files={f"files/series/mnist_sample": f"sample_{step}_label_{label}.png"},
        step=step,
    )

## Log custom string series

Neptune automatically captures standard console logs: `stdout` and `stderr`. For details, see [Console logs](https://docs.neptune.ai/console_logs) in the Neptune docs.

You can also log custom string messages using `log_string_series()`. Each message is 
associated with a step value, making it useful for tracking progress during training.
 
For example, you can log:
- Error messages
- Progress updates 
- Custom debugging information

See [logged example](https://scale.neptune.ai/o/examples/org/quickstart/runs/details?viewId=9ea24429-7823-477e-a740-aa667c66a314&detailsTab=dashboard&dashboardId=9ea2595e-90a2-412e-964a-1840955af6f6&runIdentificationKey=QUICK-10&type=run) in custom dashboard.

For instructions, see [Log text](https://docs.neptune.ai/log_text) in the Neptune docs.

In [ ]:
run.log_string_series(
    data={
        "status": "Starting training",
    },
    step=0,
)

for step in range(1, 10):
    run.log_string_series(
        data={
            "status": f"Training {step=}",
        },
        step=step,
    )

run.log_string_series(
    data={
        "status": "Training complete!",
    },
    step=10,
)

## Close the run

Once you're done logging metrics, to stop the run, use the `close()` method. This ensures that all pending operations are processed.


In [ ]:
run.close()

## Next steps

You can analyze runs by creating custom table views, dashboards, and reports.

To learn more, [see the docs](https://docs.neptune.ai/analyze_runs).

You can also explore a sample, read-only project:
- [Custom view](https://scale.neptune.ai/o/examples/org/quickstart/runs/table?viewId=9ea24429-7823-477e-a740-aa667c66a314)
- [Dashboard](https://scale.neptune.ai/o/examples/org/quickstart/runs/details?viewId=9ea24429-7823-477e-a740-aa667c66a314&detailsTab=dashboard&dashboardId=9ea2595e-90a2-412e-964a-1840955af6f6&runIdentificationKey=QUICK-10&type=run)
- [Report](https://scale.neptune.ai/o/examples/org/quickstart/reports/9ea26258-2aed-4441-9b58-bab619215f6f)